# OGGM glacier simulator

App to initialize a simple glacier and look how it develops under changing mass balance profiles.

## Import packages

import plotting libaries

In [ ]:
import holoviews as hv
from holoviews import opts
from holoviews.streams import Stream, param
import panel as pn

# is needed for unlocking bokeh doc
from panel.io import unlocked

# is needed for second x axis
from bokeh.models import Range1d, LinearAxis

hv.extension('bokeh', width=100)
pn.extension()

import OGGM packages

In [ ]:
# Constants for initialization
from functools import partial
from oggm.core.flowline import FluxBasedModel, RectangularBedFlowline, TrapezoidalBedFlowline, ParabolicBedFlowline
from oggm.core.massbalance import LinearMassBalance, MassBalanceModel
from oggm import cfg
from oggm.cfg import SEC_IN_YEAR
cfg.initialize_minimal()

# There are several solvers in OGGM core. We use the default one for this experiment
FlowlineModel = partial(FluxBasedModel, min_dt=0,
                        cfl_number=0.02)  # max_dt=cfg.SEC_IN_YEAR

and some other useful packages

In [ ]:
import numpy as np

## Define model domain

In [ ]:
# number of steps from bottem to top of glacier
nx = 200

# model grid spacing in m
map_dx = 100

# distance along glacier (x-axis of glacier profil plot) in km
distance_along_glacier = np.linspace(0, nx, nx) * map_dx * 1e-3

# glacier top height
glacier_top_height = 4000

# glacier bottom height
glacier_bottom_height = 0

## Define default values for parameters and range for menu sliders

In [ ]:
# ELA height
ELA_height_min = 1000
ELA_height_max = 3500
ELA_height_default = 3000
ELA_height_step = 100

# mass balance gradient
mb_gradient_max = 15
mb_gradient_min = 1
mb_gradient_default = 4
mb_gradient_step = 1
mb_gradient_above_equ_below = mb_gradient_min - mb_gradient_step

# years of calculation
years_min = 10
years_max = 1000
years_default = 150
years_step = 10

# linear bed rock slope
bed_rock_slope_values = ['39°', '22°', '15°', '11°', '7°', '3°']
bed_rock_slope_default = '11°'

# model dyears (how often the model state should be shown)
dyears_model_options = [1, 2, 5, 10]
dyears_model_default = 10

# maximum calculation years of model (before it gets interrupted)
max_calc_years_start = 100
max_calc_years_end = 5000
max_calc_years_step = 100
max_calc_years_default = 1000

## Define global model variables:

In [ ]:
bed_h = np.linspace(glacier_top_height, 0, nx)
mb_model = []
model = []
run_surface_heights = []
glacier_outlines = []
glen_a = cfg.PARAMS['glen_a']
fs = 0

# define width, so glacier bed could be created when plot is created (after creation model is ready)
widths = np.zeros(nx) + 4.

# years after the model progress should be shown
dyears = dyears_model_default

# years after model should be aborted
max_calc_years = max_calc_years_default

## Define multilinear MassBalanceModel

In [ ]:
class MultiLinearMassBalance(MassBalanceModel):
    """Constant mass-balance as a linear function of altitude, but with 
    different mass balance gradient below and above the ELA.
    """

    def __init__(self, ela_h, grad_below=3., grad_above=3., max_mb=None):
        """ Initialize.
        Parameters
        ----------
        ela_h: float
            Equilibrium line altitude (units: [m])
        grad_below: float
            Mass-balance gradient below ELA(unit: [mm w.e. yr-1 m-1])
        grad_above: float
            Mass-balance gradient above ELA(unit: [mm w.e. yr-1 m-1])
        max_mb: float
            Cap the mass balance to a certain value (unit: [mm w.e. yr-1])
        Attributes
        ----------
        temp_bias : float, default 0
            A "temperature bias" doesn't makes much sense in the linear MB
            context, but we implemented a simple empirical rule:
            + 1K -> ELA + 150 m
        """
        super(MultiLinearMassBalance, self).__init__()
        self.hemisphere = 'nh'
        self.valid_bounds = [-1e4, 2e4]  # in m
        self.orig_ela_h = ela_h
        self.ela_h = ela_h
        self.grad_below = grad_below
        self.grad_above = grad_above
        self.max_mb = max_mb
        self._temp_bias = 0

    @property
    def temp_bias(self):
        """Temperature bias to add to the original series."""
        return self._temp_bias

    @temp_bias.setter
    def temp_bias(self, value):
        """Temperature bias to change the ELA."""
        self.ela_h = self.orig_ela_h + value * 150
        self._temp_bias = value

    def get_monthly_mb(self, heights, **kwargs):
        heights = np.asarray(heights)
        mb = np.where(heights <= self.ela_h,
                      (heights - self.ela_h) * self.grad_below,
                      (heights - self.ela_h) * self.grad_above)
        if self.max_mb is not None:
            clip_max(mb, self.max_mb, out=mb)
        return mb / SEC_IN_YEAR / self.rho

    def get_annual_mb(self, heights, **kwargs):
        return self.get_monthly_mb(heights, **kwargs)

## Define some plotting variables

In [ ]:
# variable for the first initailisation of the plots
first_run = True

# default tools for plots
default_tools_geometry = ['save', 'box_zoom', 'reset']
default_tools_timeseries = ['pan', 'save', 'box_zoom',
                            'xwheel_zoom', 'hover', 'reset']
# is needed so xwheel_zoom is activated by default
default_active_tools_timeseries = ['xwheel_zoom']

# Variable to follow which text is currently displayed in the headerline
header_text_status = 0

# colormaps for velocity and thickness, number of colors define number of different categories
colormap_v_h = hv.plotting.util.colorcet_cmap_to_palette('coolwarm_r', 16)
colormap_h = colormap_v_h[8:]
colormap_v = colormap_v_h[:8][::-1]

# define color of glacier outline
glacier_line_color = hv.plotting.util.colorcet_cmap_to_palette(
    'coolwarm', 1)[0]

# define nan array for glacier outline plotting in width plot
nan_array = np.empty(nx)
nan_array.fill(np.nan)

# index to force a 'change' for geometry figure
geometry_figure_change_index = True

# define variables for tracking changes of glacier for timeseries plot
glacier_volume = [0]
glacier_area = [0]
glacier_length = [0]
glacier_max_velocity = [0]
glacier_max_thickness = [0]
glacier_max_ice_flux = [0]
glacier_AAR = [0]

# define variable for tracking time for timeseries plot
glacier_time = [0]

# index to force a 'change' for timeseries figure (dynamic volume and length plot)
timeseries_figure_change_index = True

# define dictionary for table
table_values = {'years': [],
                'ELA': [],
                'mb_gradient': [],
                'glen_a': [],
                'sliding': [],
                'equilibrium': [],
                'bedrock_profil': [],
                'slope': [],
                'width': [],
                'outside_domain': []}

# define dictionary for temporary table values
# is needed for the table when the model is acdvancing to save the not changed geometry
tmp_table_values = {'bedrock_profil': [],
                    'width': [],
                    'slope': []}

# index to force a 'change' for timeseries dynamic map
timeseries_table_change_index = True

# variable to store vertical lines position in volume and length plot
timeseries_vlin_pos = [0]

# for the ELA position in the width plot
ELA_x_position = 0
ELA_width = 0

# variables to calculate glacier wide mass balance
glacier_wide_trapezoidal_area = []
glacier_wide_mb = []
glacier_mb_percentage = []

# define default language
language = 'en'

# is needed when language is changed
last_info_text_stage = 'new model'

# for scale geometry plot size
geometry_figure_scale_default = 0.9
geometry_figure_scale = 1.1  # definition of holoviews

# for change of fontsize of infotext
infotext_fontsize = '10pt'

# define y limits for width plot
width_plot_y_min = -300
width_plot_y_max = 300

# define bin width for mb curve
mb_curve_bin_width = 80

## Define global curves (elements of subplots)

In [ ]:
bed_rock_height_curve = []
bed_rock_width_curve = []
glacier_height_curve = []
mb_curve = []
width_curve = []
info_text = []
timeseries_top_curve = []
timeseries_middle_curve = []
timeseries_bottom_curve = []
timeseries_table = []

## Define Headerline with help functionality

Define appearance of Headerline and also implement the help functionality, so the help text is displayed in the Headerline

In [ ]:
# import text for the headerline and the available languages
from app_text import supported_languages, header_text

### define actual help function with help text

In [ ]:
# define a number for each help page, must match number of help_function()
help_pages = {
    'start_page': 0,
    'beginner_mode': 1,
    'advanced_mode_1': 2,
    'advanced_mode_2': 3,
    'advanced_mode_3': 4,
    'model_opt_1': 5,
    'model_opt_2': 6,
    'geometry_opt_2': 7,
    'find_help_here': 8,
    'geometry_plot_1': 9,
    'geometry_plot_2': 10,
    'geometry_plot_3': 11,
    'timeseries_plot_1': 12,
    'timeseries_plot_2': 13,
    'timeseries_opt': 14,
    'geometry_opt_1': 15,
}

# define the order in which the help text should be displayed
help_page_order = np.array([
    help_pages['start_page'],
    help_pages['beginner_mode'],
    help_pages['geometry_plot_1'],
    help_pages['geometry_plot_2'],
    help_pages['geometry_plot_3'],
    help_pages['geometry_opt_1'],
    help_pages['geometry_opt_2'],
    help_pages['timeseries_plot_1'],
    help_pages['timeseries_plot_2'],
    help_pages['timeseries_opt'],
    help_pages['advanced_mode_1'],
    help_pages['advanced_mode_2'],
    help_pages['advanced_mode_3'],
    help_pages['model_opt_1'],
    help_pages['model_opt_2'],
    help_pages['find_help_here']
])

In [ ]:
def help_function():
    global header_text_status

    # to switch next and privous button clickability on/off at start and end of help
    if header_text_status == help_page_order[0]:
        previous_help_button.disabled = True
        next_help_button.disabled = False
    elif header_text_status == help_page_order[-1]:
        previous_help_button.disabled = False
        next_help_button.disabled = True
    else:
        previous_help_button.disabled = False
        next_help_button.disabled = False

    # change helptext and activate desired tab in menu
    if header_text_status == help_pages['start_page']:
        header_text_markdown.object = header_text['help_start_page'][language]

    elif header_text_status == help_pages['beginner_mode']:
        header_text_markdown.object = header_text['help_beginner_mode'][language]

        tab_menu.active = 0

    elif header_text_status == help_pages['advanced_mode_1']:
        header_text_markdown.object = header_text['help_advanced_mode_1'][language]

        tab_menu.active = 1
        advanced_panel.active = 0

    elif header_text_status == help_pages['advanced_mode_2']:
        header_text_markdown.object = header_text['help_advanced_mode_2'][language]

        tab_menu.active = 1
        advanced_panel.active = 0

    elif header_text_status == help_pages['advanced_mode_3']:
        header_text_markdown.object = header_text['help_advanced_mode_3'][language]

        tab_menu.active = 1
        advanced_panel.active = 1

    elif header_text_status == help_pages['model_opt_1']:
        header_text_markdown.object = header_text['help_model_opt_1'][language]

        tab_menu.active = 4

    elif header_text_status == help_pages['model_opt_2']:
        header_text_markdown.object = header_text['help_model_opt_2'][language]

        tab_menu.active = 4

    elif header_text_status == help_pages['geometry_opt_2']:
        header_text_markdown.object = header_text['help_geometry_opt_2'][language]

        tab_menu.active = 2

    elif header_text_status == help_pages['find_help_here']:
        header_text_markdown.object = header_text['help_find_help_here'][language]

        tab_menu.active = 5

    elif header_text_status == help_pages['geometry_plot_1']:
        header_text_markdown.object = header_text['help_geometry_plot_1'][language]

        figures.active = 0

    elif header_text_status == help_pages['geometry_plot_2']:
        header_text_markdown.object = header_text['help_geometry_plot_2'][language]

        figures.active = 0

    elif header_text_status == help_pages['geometry_plot_3']:
        header_text_markdown.object = header_text['help_geometry_plot_3'][language]

        figures.active = 0

    elif header_text_status == help_pages['timeseries_plot_1']:
        header_text_markdown.object = header_text['help_timeseries_plot_1'][language]

        figures.active = 1

    elif header_text_status == help_pages['timeseries_plot_2']:
        header_text_markdown.object = header_text['help_timeseries_plot_2'][language]

        figures.active = 1

    elif header_text_status == help_pages['timeseries_opt']:
        header_text_markdown.object = header_text['help_timeseries_opt'][language]

        tab_menu.active = 3

    elif header_text_status == help_pages['geometry_opt_1']:
        header_text_markdown.object = header_text['help_geometry_opt_1'][language]

        tab_menu.active = 2

### help functions for Headerline widgets (help navigation buttons, set language)

In [ ]:
def next_help_button_click(arg=None):
    global header_text_status

    # find index of current help page
    cur_ind = np.argwhere(help_page_order == header_text_status)

    # check to stay on last page
    if cur_ind == help_page_order.size - 1:
        cur_ind -= 1

    # set the next page number
    header_text_status = help_page_order[cur_ind + 1].item()

    help_function()


def previous_help_button_click(arg=None):
    global header_text_status

    # find index of current help page
    cur_ind = np.argwhere(help_page_order == header_text_status)

    # check to stay on first page
    if cur_ind == 0:
        cur_ind += 1

    # set the previous page number
    header_text_status = help_page_order[cur_ind - 1].item()

    help_function()

In [ ]:
def language_selector_function(arg=None):
    global language
    language = language_selector.value
    change_language()


def change_language(arg=None):
    global geometry_figure_change_index
    global timeseries_figure_change_index
    global timeseries_table_change_index

    # set language of headerline
    set_headerline_language()

    # set language of menu widgets
    set_beginner_panel_language()
    set_advanced_panel_widget_language()
    set_advanced_panel_tabs()
    set_help_panel_language()
    set_geometry_option_panel_language()
    set_timeseries_panel_language()
    set_model_option_panel_language()

    # set language of menu titles
    set_tab_menu_tabs()

    # set tab-title language of figuers tab menu
    set_figures_tabs()

    # set language of geometry plot
    set_bed_rock_curves()
    set_glacier_height_curve()
    set_width_curve()
    set_mb_curve()
    set_info_text(stage=last_info_text_stage)
    with unlocked():
        geometry_figure.event(change=geometry_figure_change_index)
    geometry_figure_change_index = not geometry_figure_change_index

    # set language of timeseries plot
    set_timeseries_bottom_curve()
    set_timeseries_middle_curve()
    set_timeseries_top_curve()
    set_timeseries_table()
    with unlocked():
        timeseries_figure.event(change=timeseries_figure_change_index)
    timeseries_figure_change_index = not timeseries_figure_change_index
    with unlocked():
        timeseries_table_DynamicMap.event(change=timeseries_table_change_index)
    timeseries_table_change_index = not timeseries_table_change_index

### define elements of Headerline

function to set Headerline language

In [ ]:
def set_headerline_language():
    next_help_button.name = header_text['next_button_text'][language]

    previous_help_button.name = header_text['previous_button_text'][language]

    # changes language of help text
    help_function()

define elements

In [ ]:
header_text_markdown = pn.pane.Markdown(
    sizing_mode='stretch_width', height=100)

next_help_button = pn.widgets.Button(disabled=False,
                                     button_type='success',
                                     width=75)
next_help_button.param.watch(next_help_button_click, 'clicks')

previous_help_button = pn.widgets.Button(disabled=True,
                                         button_type='danger',
                                         width=75)
previous_help_button.param.watch(previous_help_button_click, 'clicks')

language_selector = pn.widgets.RadioBoxGroup(name='select your language',
                                             options=supported_languages,
                                             inline=True,
                                             margin=(0, 0),
                                             width=70)
language_selector.param.watch(language_selector_function, 'value')

oggm_edu_logo = '<a href="http://edu.oggm.org"><img src="https://edu.oggm.org/en/latest/_static/logos/oggm_edu_s_alpha.png" width=170></a>'

set_headerline_language()

### Put actual Headerline together

In [ ]:
header = pn.Row(pn.Pane(oggm_edu_logo),
                pn.layout.Spacer(),
                header_text_markdown,
                pn.Column(language_selector,
                          previous_help_button,
                          next_help_button),
                height=110,
                sizing_mode='stretch_width')

## Define Panels for tab menu

Defining the individual menus and putting them together in an tab menu.

In [ ]:
# define background color of tab menu
menu_background = '#f4f4f4'

# define width for menu-tabs
panel_width = 250

# import text for panels
from app_text import panel_text, tab_menu_headings

### help function to toggle buttons

is needed so no new model run is started when there is an ongoing model run

In [ ]:
def toggle_buttons(arg=None):
    beginner_button.disabled = not beginner_button.disabled
    run_model_equilibrium_button.disabled = not run_model_equilibrium_button.disabled
    run_model_years_button.disabled = not run_model_years_button.disabled
    create_new_model_button.disabled = not create_new_model_button.disabled

### Panel for beginner mode

beginner button function

In [ ]:
def beginner_button_click(arg=None):
    # set Variables not changeable in beginner mode to there default values
    bed_rock_profile.value = panel_text['bed_rock_profil_values'][language][0]
    glens_creep_parameter.value = panel_text['glens_creep_parameter_values'][language][1]
    sliding_parameter.value = panel_text['sliding_parameter_values'][language][0]
    mb_gradient_advanced_above.value = mb_gradient_above_equ_below

    # for consitency set values of the advanced mode the same as choosen in beginner mode
    mb_gradient_advanced_below.value = mb_gradient.value
    ELA_height_advanced.value = ELA_height.value
    bed_rock_width_advanced.value = bed_rock_width.value
    bed_rock_slope_advanced.value = bed_rock_slope.value

    # set numeric values out of selection
    set_glen_a()
    set_fs()

    if toggle_buttons_checkbox.value:
        toggle_buttons()

    run_the_model(stage='beginner mode')

    if toggle_buttons_checkbox.value:
        toggle_buttons()

function so change language

In [ ]:
def set_beginner_panel_language():
    bed_rock_slope.name = panel_text['bed_rock_slope_beginner_heading'][language]

    bed_rock_width.name = panel_text['bed_rock_width_heading'][language]

    mb_gradient.name = panel_text['mb_gradient_heading'][language]

    ELA_height.name = panel_text['ELA_height_heading'][language]

    beginner_button.name = panel_text['beginner_button_heading'][language]

    bed_rock_width.options = panel_text['bed_rock_width_values'][language]

actual beginner Panel

In [ ]:
bed_rock_slope = pn.widgets.Select(options=bed_rock_slope_values,
                                   value=bed_rock_slope_default,
                                   sizing_mode='stretch_width')

bed_rock_width = pn.widgets.Select(value=panel_text['bed_rock_width_values'][language][0],
                                   sizing_mode='stretch_width')

mb_gradient = pn.widgets.IntSlider(start=mb_gradient_min,
                                   end=mb_gradient_max,
                                   step=mb_gradient_step,
                                   value=mb_gradient_default,
                                   sizing_mode='stretch_width')

ELA_height = pn.widgets.IntSlider(start=ELA_height_min,
                                  end=ELA_height_max,
                                  step=ELA_height_step,
                                  value=ELA_height_default,
                                  sizing_mode='stretch_width')

beginner_button = pn.widgets.Button(button_type='primary',
                                    sizing_mode='stretch_width')
beginner_button.param.watch(beginner_button_click, 'clicks')

set_beginner_panel_language()

beginner_panel = pn.Column(bed_rock_slope,
                           bed_rock_width,
                           mb_gradient,
                           ELA_height,
                           beginner_button,
                           background=menu_background,
                           width=panel_width,
                           sizing_mode='stretch_height')

### Panel for advanced mode

advanced button help functions

In [ ]:
def advance_button_click():
    # for consitency set values of the beginner mode the same as choosen in advanced mode
    mb_gradient.value = mb_gradient_advanced_below.value
    ELA_height.value = ELA_height_advanced.value
    bed_rock_width.value = bed_rock_width_advanced.value
    bed_rock_slope.value = bed_rock_slope_advanced.value

    # set numeric values out of selection
    set_glen_a()
    set_fs()

advanced button functions

In [ ]:
def run_model_equilibrium_button_click(arg=None):
    advance_button_click()
    if toggle_buttons_checkbox.value:
        toggle_buttons()
    run_the_model(stage='advance equilibrium')
    if toggle_buttons_checkbox.value:
        toggle_buttons()


def run_model_years_button_click(arg=None):
    advance_button_click()
    if toggle_buttons_checkbox.value:
        toggle_buttons()
    run_the_model(stage='advance years')
    if toggle_buttons_checkbox.value:
        toggle_buttons()


def create_new_model_button_click(arg=None):
    # set sliders of beginner mode to the same as choosen in advanced mode
    bed_rock_width.value = bed_rock_width_advanced.value
    bed_rock_slope.value = bed_rock_slope_advanced.value

    init_model()

help function for selecting slope when profile is linear

In [ ]:
def bed_rock_profile_change(arg=None):
    if bed_rock_profile.value == panel_text['bed_rock_profil_values'][language][0]:
        bed_rock_slope_advanced.disabled = False
    else:
        bed_rock_slope_advanced.disabled = True

function to change the language of the mass balance gradient above options

In [ ]:
def set_mb_grad_above_name(arg=None):
    name = panel_text['mb_gradient_above_heading'][language]

    if mb_gradient_advanced_above.value == mb_gradient_above_equ_below:
        name += panel_text['mb_gradient_above_equal_below'][language]
        mb_gradient_advanced_above.show_value = False
    else:
        mb_gradient_advanced_above.show_value = True

    mb_gradient_advanced_above.name = name

function to change the language

In [ ]:
def set_advanced_panel_widget_language():
    mb_gradient_advanced_below.name = panel_text['mb_gradient_below_heading'][language]

    set_mb_grad_above_name()

    bed_rock_profile.name = panel_text['bed_rock_profil_heading'][language]

    ELA_height_advanced.name = panel_text['ELA_height_heading'][language]

    bed_rock_slope_advanced.name = panel_text['bed_rock_slope_advanced_heading'][language]

    bed_rock_width_advanced.name = panel_text['bed_rock_width_heading'][language]

    glens_creep_parameter.name = panel_text['glens_creep_parameter_heading'][language]

    sliding_parameter.name = panel_text['sliding_parameter_heading'][language]

    run_model_years.name = panel_text['years_to_advance_model_heading'][language]

    run_model_equilibrium_button.name = panel_text['button_for_equilibrium_run_heading'][language]

    run_model_years_button.name = panel_text['button_to_advance_for_some_years_heading'][language]

    create_new_model_button.name = panel_text['button_to_create_new_model_heading'][language]

    bed_rock_profile.options = panel_text['bed_rock_profil_values'][language]
    bed_rock_profile.value = panel_text['bed_rock_profil_values'][language][0]

    bed_rock_width_advanced.options = panel_text['bed_rock_width_values'][language]
    bed_rock_width_advanced.value = panel_text['bed_rock_width_values'][language][0]

    glens_creep_parameter.options = panel_text['glens_creep_parameter_values'][language]
    glens_creep_parameter.value = panel_text['glens_creep_parameter_values'][language][1]

    sliding_parameter.options = panel_text['sliding_parameter_values'][language]
    sliding_parameter.value = panel_text['sliding_parameter_values'][language][0]

actual advanced Panel

In [ ]:
mb_gradient_advanced_below = pn.widgets.IntSlider(start=mb_gradient_min,
                                                  end=mb_gradient_max,
                                                  step=mb_gradient_step,
                                                  value=mb_gradient_default,
                                                  sizing_mode='stretch_width')

mb_gradient_advanced_above = pn.widgets.IntSlider(start=mb_gradient_above_equ_below,
                                                  end=mb_gradient_max,
                                                  step=mb_gradient_step,
                                                  value=mb_gradient_above_equ_below,
                                                  sizing_mode='stretch_width')
mb_gradient_advanced_above.param.watch(set_mb_grad_above_name, 'value')

bed_rock_profile = pn.widgets.Select(sizing_mode='stretch_width')
bed_rock_profile.param.watch(bed_rock_profile_change, 'value')

ELA_height_advanced = pn.widgets.IntSlider(start=ELA_height_min,
                                           end=ELA_height_max,
                                           step=ELA_height_step,
                                           value=ELA_height_default,
                                           sizing_mode='stretch_width')

bed_rock_slope_advanced = pn.widgets.Select(options=bed_rock_slope_values,
                                            value=bed_rock_slope_default,
                                            sizing_mode='stretch_width')

bed_rock_width_advanced = pn.widgets.Select(sizing_mode='stretch_width')

glens_creep_parameter = pn.widgets.Select(sizing_mode='stretch_width')

sliding_parameter = pn.widgets.RadioBoxGroup(sizing_mode='stretch_width')

run_model_years = pn.widgets.IntSlider(start=years_min,
                                       end=years_max,
                                       step=years_step,
                                       value=years_default,
                                       sizing_mode='stretch_width')

run_model_equilibrium_button = pn.widgets.Button(button_type='primary',
                                                 sizing_mode='stretch_width')
run_model_equilibrium_button.param.watch(
    run_model_equilibrium_button_click, 'clicks')

run_model_years_button = pn.widgets.Button(button_type='primary',
                                           sizing_mode='stretch_width')
run_model_years_button.param.watch(run_model_years_button_click, 'clicks')

create_new_model_button = pn.widgets.Button(button_type='primary',
                                            sizing_mode='stretch_width')
create_new_model_button.param.watch(create_new_model_button_click, 'clicks')

set_advanced_panel_widget_language()

advanced_panel = pn.Tabs(('', []),
                         ('', []),
                         background=menu_background,
                         width=panel_width,
                         tabs_location='above')

advanced_panel_first_tab = pn.Column(ELA_height_advanced,
                                     mb_gradient_advanced_below,
                                     mb_gradient_advanced_above,
                                     glens_creep_parameter,
                                     pn.Row(sliding_parameter,
                                            width=90),
                                     run_model_years,
                                     pn.Column(run_model_years_button,
                                               run_model_equilibrium_button,
                                               sizing_mode='stretch_width'),
                                     width=panel_width,
                                     sizing_mode='stretch_height')

advanced_panel_second_tab = pn.Column(bed_rock_profile,
                                      bed_rock_slope_advanced,
                                      bed_rock_width_advanced,
                                      create_new_model_button,
                                      width=panel_width,
                                      sizing_mode='stretch_height')


# is necessary to change the tabs title language
def set_advanced_panel_tabs():
    advanced_panel.clear()

    advanced_panel.append((panel_text['run_model_tab_heading'][language],
                           advanced_panel_first_tab))

    advanced_panel.append((panel_text['create_new_model_tab_heading'][language],
                           advanced_panel_second_tab))


set_advanced_panel_tabs()

### Panel for help

help functions for buttons

In [ ]:
def beginner_mode_help_button_click(arg=None):
    global header_text_status

    # choose the start page
    header_text_status = help_pages['beginner_mode']
    help_function()


def advanced_mode_help_button_click(arg=None):
    global header_text_status

    # choose the start page
    header_text_status = help_pages['advanced_mode_1']
    help_function()


def geometry_options_help_button_click(arg=None):
    global header_text_status

    # choose the start page
    header_text_status = help_pages['geometry_opt_1']
    help_function()


def timeseries_options_help_button_click(arg=None):
    global header_text_status

    # choose the start page
    header_text_status = help_pages['timeseries_opt']
    help_function()


def model_options_help_button_click(arg=None):
    global header_text_status

    # choose the start page
    header_text_status = help_pages['model_opt_1']
    help_function()


def find_help_here_help_button_click(arg=None):
    global header_text_status

    # choose the start page
    header_text_status = help_pages['find_help_here']
    help_function()


def geometry_plot_help_button_click(arg=None):
    global header_text_status

    # choose the start page
    header_text_status = help_pages['geometry_plot_1']
    help_function()


def timeseries_plot_help_button_click(arg=None):
    global header_text_status

    # choose the start page
    header_text_status = help_pages['timeseries_plot_1']
    help_function()

function to change the language

In [ ]:
def set_help_panel_language():
    help_intro_text.object = panel_text['help_panel_description'][language]

    beginner_mode_help_button.name = tab_menu_headings['beginner_mode'][language]

    advanced_mode_help_button.name = tab_menu_headings['advanced_mode'][language]

    geometry_options_help_button.name = tab_menu_headings['geometry_opt'][language]

    timeseries_options_help_button.name = tab_menu_headings['timeseries_opt'][language]

    model_options_help_button.name = tab_menu_headings['model_opt'][language]

    find_help_here_help_button.name = tab_menu_headings['find_help_here'][language]

    geometry_plot_help_button.name = tab_menu_headings['geometry_plot'][language]

    timeseries_plot_help_button.name = tab_menu_headings['timeseries_plot'][language]

actual help Panel

In [ ]:
help_intro_text = pn.pane.Markdown(
    sizing_mode='stretch_height', width=panel_width-10)

beginner_mode_help_button = pn.widgets.Button(button_type='primary',
                                              sizing_mode='stretch_width')
beginner_mode_help_button.param.watch(
    beginner_mode_help_button_click, 'clicks')

advanced_mode_help_button = pn.widgets.Button(button_type='primary',
                                              sizing_mode='stretch_width')
advanced_mode_help_button.param.watch(
    advanced_mode_help_button_click, 'clicks')

geometry_options_help_button = pn.widgets.Button(button_type='primary',
                                                 sizing_mode='stretch_width')
geometry_options_help_button.param.watch(
    geometry_options_help_button_click, 'clicks')

timeseries_options_help_button = pn.widgets.Button(button_type='primary',
                                                   sizing_mode='stretch_width')
timeseries_options_help_button.param.watch(
    timeseries_options_help_button_click, 'clicks')

model_options_help_button = pn.widgets.Button(button_type='primary',
                                              sizing_mode='stretch_width')
model_options_help_button.param.watch(
    model_options_help_button_click, 'clicks')

find_help_here_help_button = pn.widgets.Button(button_type='primary',
                                               sizing_mode='stretch_width')
find_help_here_help_button.param.watch(
    find_help_here_help_button_click, 'clicks')

geometry_plot_help_button = pn.widgets.Button(button_type='primary',
                                              sizing_mode='stretch_width')
geometry_plot_help_button.param.watch(
    geometry_plot_help_button_click, 'clicks')

timeseries_plot_help_button = pn.widgets.Button(button_type='primary',
                                                sizing_mode='stretch_width')
timeseries_plot_help_button.param.watch(
    timeseries_plot_help_button_click, 'clicks')

set_help_panel_language()

help_panel = pn.Column(pn.Row(beginner_mode_help_button,
                              sizing_mode='stretch_width'),
                       pn.Row(advanced_mode_help_button,
                              sizing_mode='stretch_width'),
                       pn.Row(timeseries_options_help_button,
                              sizing_mode='stretch_width'),
                       pn.Row(geometry_options_help_button,
                              sizing_mode='stretch_width'),
                       pn.Row(model_options_help_button,
                              sizing_mode='stretch_width'),
                       pn.Row(geometry_plot_help_button,
                              sizing_mode='stretch_width'),
                       pn.Row(timeseries_plot_help_button,
                              sizing_mode='stretch_width'),
                       pn.Row(find_help_here_help_button,
                              sizing_mode='stretch_width'),
                       help_intro_text,
                       background=menu_background,
                       width=panel_width,
                       sizing_mode='stretch_height')

### Panel for geometry options

help functions

In [ ]:
def show_velocity_changed(arg=None):
    global geometry_figure_change_index
    set_glacier_height_curve()
    with unlocked():
        geometry_figure.event(change=geometry_figure_change_index)
    geometry_figure_change_index = not geometry_figure_change_index


def show_thickness_changed(arg=None):
    global geometry_figure_change_index
    set_width_curve()
    with unlocked():
        geometry_figure.event(change=geometry_figure_change_index)
    geometry_figure_change_index = not geometry_figure_change_index


def change_mb_curve_bin_width(arg=None):
    global mb_curve_bin_width
    global geometry_figure_change_index

    mb_curve_bin_width = mb_curve_bin_width_slider.value
    set_mb_curve()

    # make change visible
    set_info_text(stage=last_info_text_stage)
    with unlocked():
        geometry_figure.event(change=geometry_figure_change_index)
    geometry_figure_change_index = not geometry_figure_change_index


def change_figure_size(arg=None):
    global geometry_figure_scale
    global geometry_figure_change_index

    # calculation is necessary because of definition of aspect ratio of holoviews objects
    geometry_figure_scale = 2 - geometry_plot_figure_size.value

    # make change visible
    with unlocked():
        geometry_figure.event(change=geometry_figure_change_index)
    geometry_figure_change_index = not geometry_figure_change_index


def change_infotext_fontsize(arg=None):
    global infotext_fontsize
    global geometry_figure_change_index

    infotext_fontsize = str(infotext_fontsize_slider.value) + 'pt'

    # make change visible
    set_info_text(stage=last_info_text_stage)
    with unlocked():
        geometry_figure.event(change=geometry_figure_change_index)
    geometry_figure_change_index = not geometry_figure_change_index

function to change the language

In [ ]:
def set_geometry_option_panel_language():
    show_velocity.name = panel_text['show_velocity_heading'][language]

    show_thickness.name = panel_text['show_thickness_heading'][language]

    mb_curve_bin_width_slider.name = panel_text['mb_curve_bin_width_heading'][language]

    geometry_plot_figure_size.name = panel_text['geometry_figure_size_heading'][language]

    infotext_fontsize_slider.name = panel_text['geometry_figure_fontsize'][language]

    show_geometry_plot.name = panel_text['show_geometry_plot_heading'][language]

actual panel

In [ ]:
mb_curve_bin_width_slider = pn.widgets.IntSlider(start=30,
                                                 end=500,
                                                 value=80)
mb_curve_bin_width_slider.param.watch(change_mb_curve_bin_width, 'value')

show_velocity = pn.widgets.Checkbox(sizing_mode='stretch_width',
                                    )
show_velocity.param.watch(show_velocity_changed, 'value')

show_thickness = pn.widgets.Checkbox(sizing_mode='stretch_width',
                                     )
show_thickness.param.watch(show_thickness_changed, 'value')


# is needed to directly get to bokeh styling options
def colorbar_hook(plot, element):
    plot.handles['plot'].border_fill_color = menu_background
    plot.handles['plot'].min_border_right = 10
    plot.handles['plot'].min_border_bottom = 0
    plot.handles['colorbar'].background_fill_color = menu_background
    plot.handles['colorbar'].height = 7


ColorBar_v = hv.Polygons({'x': [0],
                          'y': [0],
                          'value': [0]},
                         vdims='value').opts(hooks=[colorbar_hook],
                                             responsive=True,
                                             colorbar_position='top',
                                             yaxis=None,
                                             xaxis=None,
                                             # show_frame=False,
                                             toolbar=None,
                                             frame_height=0,
                                             cmap=colormap_v,
                                             colorbar=True,
                                             colorbar_opts={'orientation': 'horizontal',
                                                            'major_label_overrides': {-1: '0',
                                                                                      -0.5: '1/4',
                                                                                      0: '1/2',
                                                                                      0.5: '3/4',
                                                                                      1: 'Max'
                                                                                      },
                                                            'major_label_text_font_size': '10pt'})

ColorBar_h = hv.Polygons({'x': [0],
                          'y': [0],
                          'value': [0]},
                         vdims='value').opts(hooks=[colorbar_hook],
                                             responsive=True,
                                             colorbar_position='top',
                                             yaxis=None,
                                             xaxis=None,
                                             # show_frame=False,
                                             toolbar=None,
                                             frame_height=0,
                                             cmap=colormap_h,
                                             colorbar=True,
                                             colorbar_opts={'orientation': 'horizontal',
                                                            'major_label_overrides': {-1: '0',
                                                                                      -0.5: '1/4',
                                                                                      0: '1/2',
                                                                                      0.5: '3/4',
                                                                                      1: 'Max'
                                                                                      },
                                                            'major_label_text_font_size': '10pt'})

geometry_plot_figure_size = pn.widgets.FloatSlider(start=0.3,
                                                   end=1.1,
                                                   step=0.1,
                                                   value=geometry_figure_scale_default)
geometry_plot_figure_size.param.watch(change_figure_size, 'value')

infotext_fontsize_slider = pn.widgets.IntSlider(start=5,
                                                end=15,
                                                value=10)
infotext_fontsize_slider.param.watch(change_infotext_fontsize, 'value')

show_geometry_plot = pn.widgets.Checkbox(value=True)

set_geometry_option_panel_language()

geometry_option_panel = pn.Column(mb_curve_bin_width_slider,
                                  show_velocity,
                                  pn.Row(ColorBar_v,
                                         width=panel_width),
                                  show_thickness,
                                  pn.Row(ColorBar_h,
                                         width=panel_width),
                                  show_geometry_plot,
                                  geometry_plot_figure_size,
                                  infotext_fontsize_slider,
                                  background=menu_background,
                                  width=panel_width,
                                  sizing_mode='stretch_height')

### Panel for timeseries plot

function to show change of choosed timeseries variable immediately

In [ ]:
def show_timeseries_change(arg=None):
    global timeseries_figure_change_index

    set_timeseries_top_curve()
    set_timeseries_middle_curve()
    set_timeseries_bottom_curve()

    # force a refresh of the timeseries plot
    with unlocked():
        timeseries_figure.event(change=timeseries_figure_change_index)
    timeseries_figure_change_index = not timeseries_figure_change_index

function to change the language

In [ ]:
def set_timeseries_panel_language():
    timeseries_top.name = panel_text['top_axis_heading'][language]

    timeseries_middle.name = panel_text['middle_axis_heading'][language]

    timeseries_bottom.name = panel_text['bottom_axis_heading'][language]

    timeseries_top.options = panel_text['timeseries_axis_options'][language]
    timeseries_middle.options = panel_text['timeseries_axis_options'][language]
    timeseries_bottom.options = panel_text['timeseries_axis_options'][language]

    timeseries_top.value = panel_text['timeseries_axis_options'][language][0]
    timeseries_middle.value = panel_text['timeseries_axis_options'][language][1]
    timeseries_bottom.value = panel_text['timeseries_axis_options'][language][2]

In [ ]:
timeseries_top = pn.widgets.Select(sizing_mode='stretch_width')

timeseries_middle = pn.widgets.Select(sizing_mode='stretch_width')

timeseries_bottom = pn.widgets.Select(sizing_mode='stretch_width')

set_timeseries_panel_language()

# functions watch value and call function 'show_timeseries_change' if value change
timeseries_top.param.watch(show_timeseries_change, 'value')
timeseries_middle.param.watch(show_timeseries_change, 'value')
timeseries_bottom.param.watch(show_timeseries_change, 'value')

# actual panel
timeseries_panel = pn.Column(timeseries_top,
                             timeseries_middle,
                             timeseries_bottom,
                             background=menu_background,
                             width=panel_width,
                             sizing_mode='stretch_height')

### Panel for model options

function to change the language

In [ ]:
def set_model_option_panel_language():
    global current_model_options_text

    stop_if_outside_domain.name = panel_text['stop_if_outside_heading'][language]

    toggle_buttons_checkbox.name = panel_text['toggle_buttons_checkbox_heading'][language]

    toggle_buttons_button.name = panel_text['toggle_buttons_button_heading'][language]

    dyears_model_slider.name = panel_text['dyears_model_slider_heading'][language]

    max_calc_years_slider.name = panel_text['max_calc_years_slider_heading'][language]

actual panel

In [ ]:
stop_if_outside_domain = pn.widgets.Checkbox(value=True,
                                             sizing_mode='stretch_width')

toggle_buttons_checkbox = pn.widgets.Checkbox(value=True,
                                              sizing_mode='stretch_width')

toggle_buttons_button = pn.widgets.Button(button_type='primary',
                                          sizing_mode='stretch_width')
toggle_buttons_button.param.watch(toggle_buttons, 'clicks')

dyears_model_slider = pn.widgets.Select(
    options=dyears_model_options, value=dyears_model_default)

max_calc_years_slider = pn.widgets.IntSlider(start=max_calc_years_start,
                                             end=max_calc_years_end,
                                             step=max_calc_years_step,
                                             value=max_calc_years_default,
                                             sizing_mode='stretch_width')

set_model_option_panel_language()

model_option_panel = pn.Column(stop_if_outside_domain,
                               toggle_buttons_checkbox,
                               toggle_buttons_button,
                               dyears_model_slider,
                               max_calc_years_slider,
                               background=menu_background,
                               width=panel_width,
                               sizing_mode='stretch_height')

### add all panels together as tab menu

In [ ]:
tab_menu = pn.Tabs(('', []),
                   height=420,
                   width=370,
                   background=menu_background,
                   tabs_location='left')


# is necassary to change the tabs title language
def set_tab_menu_tabs():
    tab_menu.clear()

    tab_menu.append((tab_menu_headings['beginner_mode'][language],
                     beginner_panel))

    tab_menu.append((tab_menu_headings['advanced_mode'][language],
                     advanced_panel))

    tab_menu.append((tab_menu_headings['geometry_opt'][language],
                     geometry_option_panel))

    tab_menu.append((tab_menu_headings['timeseries_opt'][language],
                     timeseries_panel))

    tab_menu.append((tab_menu_headings['model_opt'][language],
                     model_option_panel))

    tab_menu.append((tab_menu_headings['find_help_here'][language],
                     help_panel))


set_tab_menu_tabs()

### add logos underneath menu

In [ ]:
hv_logo = '<a href="https://holoviz.org"><img src="https://holoviz.org/assets/holoviz-logo-stacked.svg" width=80></a>'
fk_logo = '<a href="https://www.uibk.ac.at/foerderkreis1669/"><img src="https://raw.githubusercontent.com/OGGM/world-glacier-explorer/master/img/logo_1669.png" width=180 height=79></a>'

menu = pn.Column(tab_menu,
                 pn.Row(pn.Spacer(width=10),
                        pn.Pane(fk_logo),
                        pn.Spacer(width=10),
                        pn.Pane(hv_logo)))

## Define functions to set model parameters

Functions to set model parameters, which are needed to initialize a new model

### set bed h

In [ ]:
def set_bed_h():
    global bed_h
    global tmp_table_values

    # look which bed rock profile was chosen
    if bed_rock_profile.value is panel_text['bed_rock_profil_values'][language][0]:
        tmp_table_values['bedrock_profil'] = 0
        # look which slope was choosen and set the heigth at each point along the glacier
        if bed_rock_slope.value == '39°':
            bed_h = np.append(np.linspace(glacier_top_height, 0, np.int(np.round(nx / 4))),
                              np.zeros(nx - np.int(np.round(nx / 4))))

            tmp_table_values['slope'] = '39°'

        elif bed_rock_slope.value == '22°':
            bed_h = np.append(np.linspace(glacier_top_height, 0, np.int(np.round(nx / 4) * 2)),
                              np.zeros(nx - np.int(np.round(nx / 4) * 2)))

            tmp_table_values['slope'] = '22°'

        elif bed_rock_slope.value == '15°':
            bed_h = np.append(np.linspace(glacier_top_height, 0, np.int(np.round(nx / 4) * 3)),
                              np.zeros(nx - np.int(np.round(nx / 4) * 3)))

            tmp_table_values['slope'] = '15°'

        elif bed_rock_slope.value == '11°':
            bed_h = np.linspace(glacier_top_height, 0, nx)

            tmp_table_values['slope'] = '11°'

        elif bed_rock_slope.value == '7°':
            bed_h = np.linspace(glacier_top_height, 1500, nx)

            tmp_table_values['slope'] = '7°'

        elif bed_rock_slope.value == '3°':
            bed_h = np.linspace(glacier_top_height, 2900, nx)

            tmp_table_values['slope'] = '3°'

    elif bed_rock_profile.value is panel_text['bed_rock_profil_values'][language][1]:
        bed_h = np.array([(np.log(x / nx + 0.03) / np.log(0.5) * 0.2 + 0.03) *
                          glacier_top_height for x in np.arange(nx)])

        tmp_table_values['slope'] = ' '
        tmp_table_values['bedrock_profil'] = 1

    elif bed_rock_profile.value is panel_text['bed_rock_profil_values'][language][2]:
        bed_h = np.array([(np.log(-x / nx / 4 + 0.26) * 0.2 / np.log(2) + 1.3) *
                          glacier_top_height for x in np.arange(nx)])

        tmp_table_values['slope'] = ' '
        tmp_table_values['bedrock_profil'] = 2

    elif bed_rock_profile.value is panel_text['bed_rock_profil_values'][language][3]:
        # define extend of cliff
        cliff_top = 2500
        cliff_bottom = 2400

        bed_h = np.concatenate((np.linspace(glacier_top_height, 2500, np.int(nx/2)),
                                np.linspace(2400, glacier_bottom_height, np.int(nx/2))))

        tmp_table_values['slope'] = ' '
        tmp_table_values['bedrock_profil'] = 3

### set widths

In [ ]:
def set_widths():
    global widths
    global tmp_table_values

    # look which glacier width was selected
    if bed_rock_width.value is panel_text['bed_rock_width_values'][language][0]:
        widths = np.zeros(nx) + 4.

        tmp_table_values['width'] = 0

    elif bed_rock_width.value is panel_text['bed_rock_width_values'][language][1]:
        widths = np.array([(-2 / nx * x + 2.5) * 2 for x in np.arange(nx)])

        tmp_table_values['width'] = 1

    elif bed_rock_width.value is panel_text['bed_rock_width_values'][language][2]:
        widths = np.array([(2 / nx * x + 0.5) * 2 for x in np.arange(nx)])

        tmp_table_values['width'] = 2

    elif bed_rock_width.value is panel_text['bed_rock_width_values'][language][3]:
        widths = np.append(np.ones(np.int(np.round(nx/5))) * 4,
                           np.ones(np.int(nx - np.round(nx/5))) * 2)

        tmp_table_values['width'] = 3

    # set length in m
    widths = widths * map_dx

### set glen a

In [ ]:
def set_glen_a():
    global glen_a

    if glens_creep_parameter.value is panel_text['glens_creep_parameter_values'][language][1]:
        glen_a = cfg.PARAMS['glen_a']
    elif glens_creep_parameter.value is panel_text['glens_creep_parameter_values'][language][0]:
        glen_a = cfg.PARAMS['glen_a'] / 10
    elif glens_creep_parameter.value is panel_text['glens_creep_parameter_values'][language][2]:
        glen_a = cfg.PARAMS['glen_a'] * 10

### set fs

In [ ]:
def set_fs():
    global fs

    if sliding_parameter.value is panel_text['sliding_parameter_values'][language][0]:
        fs = 0
    elif sliding_parameter.value is panel_text['sliding_parameter_values'][language][1]:
        fs = 5.7e-20

### set mb model

In [ ]:
def set_mb_model():
    global mb_model

    ELA = int(ELA_height.value)

    # check if two gradients should be used for the mass balance model
    if mb_gradient_advanced_above.value == mb_gradient_above_equ_below:
        gradient = int(mb_gradient.value)
        mb_model = LinearMassBalance(ELA, grad=gradient)
    else:
        grad_below = int(mb_gradient_advanced_below.value)
        grad_above = int(mb_gradient_advanced_above.value)
        mb_model = MultiLinearMassBalance(
            ELA, grad_below=grad_below, grad_above=grad_above)

### set glacier wide trapezoidal mb area

This trapezoidal area is used for the interpolation of area between the area bins in the mass balance plot (get a 'smoother' curve, see https://github.com/OGGM/glacier_simulator/issues/13)

In [ ]:
def set_glacier_wide_trapezoidal_area():
    global glacier_wide_trapezoidal_area

    glacier_wide_trapezoidal_area = map_dx * (widths[:-1] + widths[1:]) / 2

### set glacier wide mb and percentage

In [ ]:
def set_glacier_wide_mb():
    global glacier_wide_mb
    global glacier_mb_percentage

    bin_mb = model.fls[0].bin_area_m2 * \
        model.get_mb(model.fls[0].surface_h) * cfg.SEC_IN_YEAR
    glacier_wide_mb = np.sum(bin_mb) / model.fls[0].area_m2

    total_mb_gain = np.sum(bin_mb[bin_mb >= 0])
    glacier_mb_percentage = bin_mb / total_mb_gain * 100

### calculate AAR

In [ ]:
def calculate_AAR():
    global model
    global mb_model

    total_area = model.fls[0].area_m2
    accumulation_area = np.sum(np.where(model.fls[0].surface_h > mb_model.ela_h,
                                        model.fls[0].bin_area_m2,
                                        0))

    return accumulation_area/total_area

## Define functions to initialize a model and plots

Functions initialize a new model (set all parameters and define new `FlowlineModel`) and also set all plots to there default appearance

### init model

In [ ]:
def init_model():
    global model
    global first_run

    # set all parameters for the model according to the choosen parameters from the menu
    set_mb_model()
    set_bed_h()
    set_widths()
    set_glen_a()
    set_fs()

    # set values for area interpolation in mass balance plot
    set_glacier_wide_trapezoidal_area()

    # set glacier surface height to bed height because at the beginning there is no glacier
    surface_h = bed_h

    # initialize flowline with rectangular shape of glacier
    model_flowline = RectangularBedFlowline(surface_h=surface_h, bed_h=bed_h,
                                            widths=widths/map_dx, map_dx=map_dx)

    # initialize the actual model
    model = FlowlineModel(model_flowline, mb_model=mb_model, y0=0., glen_a=glen_a,
                          fs=fs, check_for_boundaries=False, mb_elev_feedback='always')

    # initialize all plots with new model
    init_geometry_plot()
    init_timeseries_plot()

    # after first run (or any other run) set first run to false
    first_run = False

### init geometry plot

In [ ]:
def init_geometry_plot():
    global run_surface_heights
    global glacier_outlines
    global geometry_figure_change_index
    global glacier_mb_percentage

    # set surface height for plotting, at beginn old and new one are the same
    run_surface_heights = np.repeat(
        np.array(model.fls[-1].surface_h, ndmin=2), 2, axis=0)

    # set glacier outlines for plotting, at beginn old and new one are the same
    glacier_outlines = np.repeat(get_glacier_outline(), 2, axis=0)

    # set percentage of massbalance to zero at beginning
    glacier_mb_percentage = np.zeros(np.size(run_surface_heights[1]) - 1)

    # set all subplots for the geometry plot
    if show_geometry_plot.value:
        set_info_text(stage='new model')
    else:
        set_info_text(stage='switched off')
    set_bed_rock_curves()
    set_glacier_height_curve()
    set_mb_curve()
    set_width_curve()

    # force to refresh the geometry plot when not the first run, is needed for the first initialization,
    # when the dynamic map do not yet exist
    if not first_run:
        with unlocked():
            geometry_figure.event(change=geometry_figure_change_index)
        geometry_figure_change_index = not geometry_figure_change_index

### init timeseries plot

In [ ]:
def init_timeseries_plot():
    global glacier_volume
    global glacier_area
    global glacier_length
    global glacier_max_thickness
    global glacier_max_velocity
    global glacier_max_ice_flux
    global glacier_AAR
    global glacier_time
    global timeseries_vlin_pos
    global table_values
    global timeseries_table_change_index
    global timeseries_figure_change_index

    # set parameters for volume and length plot back to default
    glacier_volume = [0]
    glacier_area = [0]
    glacier_length = [0]
    glacier_max_velocity = [0]
    glacier_max_thickness = [0]
    glacier_max_ice_flux = [0]
    glacier_AAR = [0]
    glacier_time = [0]
    timeseries_vlin_pos = [0]

    # change the volume and length subplots
    set_timeseries_top_curve()
    set_timeseries_middle_curve()
    set_timeseries_bottom_curve()

    # empty table entries
    table_values = {'years': [],
                    'ELA': [],
                    'mb_gradient': [],
                    'glen_a': [],
                    'sliding': [],
                    'equilibrium': [],
                    'bedrock_profil': [],
                    'slope': [],
                    'width': [],
                    'outside_domain': []}

    # set the new values in the table
    set_timeseries_table()

    # force to refresh the geometry plot when not the first run, is needed for the first initialization,
    # when the dynamic map do not yet exist
    if not first_run:
        with unlocked():
            timeseries_table_DynamicMap.event(
                change=timeseries_table_change_index)
        timeseries_table_change_index = not timeseries_table_change_index

    # force to refresh the geometry plot when not the first run, is needed for the first initialization,
    # when the dynamic map do not yet exist
    if not first_run:
        with unlocked():
            timeseries_figure.event(change=timeseries_figure_change_index)
        timeseries_figure_change_index = not timeseries_figure_change_index

## Define function for creating curves

Functions to determine the look of each subplot or subplot element. After the definition the Functions are also called so all subplots are set to the default looking.

In [ ]:
from app_text import geometry_plot_labels

### help functions for curves

Necessary to make ELA responsive to glacier surface height changes

In [ ]:
def set_ELA_x_position_and_width():
    global ELA_x_position
    global ELA_width

    # find index for nearest grid point
    higher_index = np.where(
        run_surface_heights[1] >= ELA_height.value, 0, 1).argmax() - 1
    lower_index = np.where(
        run_surface_heights[1] >= ELA_height.value, 0, 1).argmax()

    # calculate x positon of ELA
    h = run_surface_heights[1][higher_index] - \
        run_surface_heights[1][lower_index]
    dh = run_surface_heights[1][higher_index] - ELA_height.value
    x = distance_along_glacier[lower_index] - \
        distance_along_glacier[higher_index]
    dx = x * dh / h
    ELA_x_position = distance_along_glacier[higher_index] + dx

    # calculate width at x position of ELA
    w1 = widths[higher_index]
    w2 = widths[lower_index]
    if w1 < w2:
        ELA_width = w1 + (w2 - w1) * dx / x
    else:
        ELA_width = w2 + (w1 - w2) * (x - dx) / x

### get glacier outline

In [ ]:
def get_glacier_outline():
    # set glacier outlines where glacier has thickness larger zero
    outline = np.where(model.fls[-1].thick > 0, widths, nan_array)

    # include also first point with zero glacier thickness if there is a glacier
    # and if glacier is not starting to run outside of the domain
    if (np.sum(model.fls[-1].thick > 0) > 0) & (np.sum(model.fls[-1].thick > 0) < nx):
        index = np.argwhere(np.isnan(outline)).transpose().squeeze()[0]
        outline[index] = widths[index]

    return np.array(outline, ndmin=2)

### set bed rock curves

In [ ]:
def set_bed_rock_curves():
    global bed_rock_height_curve
    global bed_rock_width_curve

    # create curve of bed rock for view from beside (top left subfigure, geometry plot)
    x_label = geometry_plot_labels['bedrock_x_label'][language]
    y_label = geometry_plot_labels['bedrock_height_y_label'][language]
    bed_rock_height_curve = hv.Area((distance_along_glacier, bed_h),
                                    x_label, y_label,
                                    ).opts(default_tools=default_tools_geometry,
                                           color='lightgray',
                                           line_alpha=0)

    # create curve of bed rock for view from above (bottom left subfigure, geometry plot)
    x_label = geometry_plot_labels['bedrock_x_label'][language]
    y_label = geometry_plot_labels['bedrock_width_y_label'][language]
    bed_rock_width_curve = hv.Area((distance_along_glacier, -widths/2, widths/2),
                                   kdims=[x_label],
                                   vdims=[y_label, 'y2']
                                   # label='glacier rock bed'
                                   ).opts(default_tools=default_tools_geometry,
                                          color='lightgray',
                                          line_alpha=0)
    bed_rock_width_curve.opts(ylim=(width_plot_y_min, width_plot_y_max))

### set glacier height curve

In [ ]:
def set_glacier_height_curve():
    global glacier_height_curve

    ELA = int(ELA_height.value)

    # set labels according to language
    x_label = geometry_plot_labels['bedrock_x_label'][language]
    y_label = geometry_plot_labels['bedrock_height_y_label'][language]
    glacier_label = geometry_plot_labels['glacier_label'][language]

    if not np.array_equal(run_surface_heights[0], bed_h):
        old_glacier = (distance_along_glacier, run_surface_heights[0])
        new_glacier_alpha = 0.6
    else:
        old_glacier = ([], [])
        new_glacier_alpha = 1

    glacier_height_curve = (hv.Area(old_glacier,
                                    x_label, y_label,
                                    # label='last'
                                    ).opts(default_tools=default_tools_geometry,
                                           line_dash='dashed',
                                           line_color='darkgray',
                                           color='white',
                                           alpha=0.6,
                                           line_alpha=1,
                                           line_width=3) *
                            hv.Area((distance_along_glacier, run_surface_heights[1]),
                                        x_label, y_label,
                                        label=glacier_label
                                    ).opts(default_tools=default_tools_geometry,
                                           color='white',
                                           line_color=glacier_line_color,
                                           alpha=new_glacier_alpha,
                                           line_alpha=1,
                                           line_width=3))

    # add ELA to plot, need start point to make ELA responsive to glacier surface height
    set_ELA_x_position_and_width()
    ELA_start_point = ELA_x_position
    ELA_end_point = np.max(distance_along_glacier)
    glacier_height_curve *= hv.Curve(([ELA_start_point, ELA_end_point], np.repeat(ELA, 2)),
                                     x_label, y_label,
                                     ).opts(default_tools=default_tools_geometry,
                                            line_dash='dashed',
                                            line_width=3,
                                            color='black')

    # looking for potential segments which can be colored if wanted
    index_of_segments = np.arange(np.sum(model.fls[-1].thick > 0))

    # show ice velocity in colors if wanted and if there is something to show
    if show_velocity.value and index_of_segments.size != 0:
        # calculate the mean velocity between two segments
        plot_velocity = [(np.array(model.get_diagnostics()['ice_velocity'])[i] +
                          np.array(model.get_diagnostics()['ice_velocity'])[i + 1]) / 2
                         for i in index_of_segments]

        # define number of velocity categories
        nr_categories = np.size(colormap_v)

        # find boarders of velocity categories
        velocity_categories = np.linspace(np.min(plot_velocity),
                                          np.max(plot_velocity),
                                          nr_categories + 1,
                                          endpoint=True)

        # create array with categories number
        segment_category = np.zeros(np.size(index_of_segments))
        for category in np.arange(nr_categories):
            segment_category = np.where((plot_velocity >= velocity_categories[category]) &
                                        (plot_velocity <=
                                         velocity_categories[category + 1]),
                                        category + 1,
                                        segment_category)

        # get arguments where one category is changing to another,
        # + 1 to always get the first argument of a new category
        category_change_arg = np.argwhere(np.diff(segment_category)
                                          ).transpose().squeeze() + 1

        # add first and last argument
        category_change_arg = np.append(0, category_change_arg)
        category_change_arg = np.append(
            category_change_arg, np.size(index_of_segments))

        # define number of polygons to draw
        nr_polygons = np.size(category_change_arg) - 1

        # define x and y values of polygons and assign a first velocity to whole polygon
        x_polygons = np.empty(nr_polygons, dtype=object)
        y_polygons = np.empty(nr_polygons, dtype=object)
        velocity_polygons = np.empty(nr_polygons)

        for polygon in np.arange(nr_polygons):
            index = np.arange(category_change_arg[polygon],
                              category_change_arg[polygon + 1] + 1)
            index_r = np.arange(category_change_arg[polygon + 1],
                                category_change_arg[polygon] - 1,
                                -1)

            x_polygons[polygon] = np.append(distance_along_glacier[index],
                                            distance_along_glacier[index_r])

            y_polygons[polygon] = np.append(bed_h[index],
                                            run_surface_heights[1][index_r])

            velocity_polygons[polygon] = segment_category[category_change_arg[polygon]]

        glacier_height_curve *= hv.Polygons([{'x': x_polygons[i],
                                              'y': y_polygons[i],
                                              'velocity': velocity_polygons[i]}
                                             for i in np.arange(nr_polygons)],
                                            vdims='velocity').opts(default_tools=default_tools_geometry,
                                                                   line_color=None,
                                                                   cmap=colormap_v)

    else:
        # when there is no glacier or colors should not be shown
        # this is needed so bokeh knows about the Polygons element
        glacier_height_curve *= hv.Polygons([{'x': [0],
                                              'y': [0],
                                              'velocity': [0]}],
                                            vdims='velocity').opts(default_tools=default_tools_geometry)

### set mb curve

help functions for calculation of subarea for area bins (see https://github.com/OGGM/glacier_simulator/issues/13)

In [ ]:
def calc_trapezodidal_subarea(w1, w2, H, h):
    return w1 * h - h**2 / H * (w1 - w2)

In [ ]:
def convert_height_diff_to_x_diff(dh, dx, h):
    return dx / dh * h

help function for second x_axis

In [ ]:
def mb_curve_second_x_axis(plot, element):
    ''' 
    A hook to put data on secondary x axis
    '''
    p = plot.state

    # to switch label language
    x2_label = geometry_plot_labels['mb_x_label'][language]

    # create secondary range and axis
    if 'twinx' not in [t for t in p.extra_x_ranges]:
        p.x_range = Range1d(start=-20, end=20)
        p.x_range.name = 'default'
        p.extra_x_ranges = {"twinx": Range1d(start=-20, end=20)}
        p.add_layout(LinearAxis(axis_label=x2_label,
                                x_range_name="twinx"), 'above')

    # set glyph y_range_name to the one we've just created
    glyph = p.renderers[-1]
    glyph.x_range_name = 'twinx'

    # set proper range
    glyph = p.renderers[-1]
    # ugly hardcoded solution
    vals = glyph.data_source.data[x2_label]
    p.extra_x_ranges["twinx"].start = vals.min() * 0.99
    p.extra_x_ranges["twinx"].end = vals.max() * 1.01

actual function for setting mb plot

In [ ]:
def set_mb_curve():
    global mb_curve

    ELA = int(ELA_height.value)

    # set labels according to language
    x1_label = geometry_plot_labels['area_x_label'][language]
    x2_label = geometry_plot_labels['mb_x_label'][language]
    y_label = geometry_plot_labels['bedrock_height_y_label'][language]
    mass_balance_label = geometry_plot_labels['mb_label'][language]

    # define the range of the mass balance curve wich should be shown
    x_range_of_mb_curve = np.array([-20, 20])
    # include ELA for multilinear mass balance
    y_range_of_mb_curve = np.array([0, ELA, 4500])

    # set min bin_width of histogram, max_bin_width = 2* min_binwidth
    min_bin_width = mb_curve_bin_width

    # get the annual mass balance for the given range of the x axis
    annual_mb = mb_model.get_annual_mb(y_range_of_mb_curve) * SEC_IN_YEAR

    # if there is a glacier draw histogram for area distribution
    if not np.array_equal(run_surface_heights[1], bed_h):
        # mass gain height difference
        mass_gain_max_height_index = np.where(
            glacier_mb_percentage > 0, glacier_mb_percentage, 0).argmax()
        mass_gain_max_height = np.ceil(
            run_surface_heights[1][mass_gain_max_height_index])
        mass_gain_delta_h = mass_gain_max_height - ELA

        # mass loss height difference
        mass_loss_min_height_index = np.where(
            glacier_mb_percentage < 0, glacier_mb_percentage, 0).argmin()
        mass_loss_min_height = np.floor(
            run_surface_heights[1][mass_loss_min_height_index])
        if mass_loss_min_height < ELA:
            mass_loss_delta_h = ELA - mass_loss_min_height
        else:
            mass_loss_delta_h = 0

        # find bin_width for gain area
        # min_bin_width < bin_width < 2 * min_bin_width
        mass_gain_bin_nr = int(np.floor(mass_gain_delta_h / min_bin_width))
        add_to_all = np.mod(mass_gain_delta_h,
                            min_bin_width) / mass_gain_bin_nr
        bin_widths_mass_gain = np.ones(
            mass_gain_bin_nr) * min_bin_width + add_to_all

        # check if sum of calculated bin_widths equals total height difference
        assert np.isclose(np.sum(bin_widths_mass_gain), mass_gain_delta_h)

        # find bin_widths for loss area
        # use the bin width of gain area to better compare areas,
        # and last bin is residual width
        if mass_loss_delta_h != 0:
            mass_loss_bin_width = bin_widths_mass_gain[0]
            mass_loss_bin_nr = int(
                np.floor(mass_loss_delta_h / mass_loss_bin_width))
            bin_widths_mass_loss = np.ones(
                mass_loss_bin_nr) * mass_loss_bin_width
            residual_mass_loss_bin = np.mod(
                mass_loss_delta_h, mass_loss_bin_width)
            if residual_mass_loss_bin != 0:
                bin_widths_mass_loss = np.append(
                    bin_widths_mass_loss, residual_mass_loss_bin)
                mass_loss_bin_nr += 1

            # check if sum of calculated bin_widths equals total height difference
            assert np.isclose(np.sum(bin_widths_mass_loss), mass_loss_delta_h)
        else:
            bin_widths_mass_loss = []
            mass_loss_bin_nr = 0

        # but all bin widths together
        bin_widths = np.append(bin_widths_mass_gain, bin_widths_mass_loss)

        # calculate actual heights for each bin
        bin_heights = np.ones(1) * mass_gain_max_height
        for i in np.arange(len(bin_widths), dtype=np.int):
            bin_heights = np.append(
                bin_heights, bin_heights[-1] - bin_widths[i])

        # calculate area and mass balance contribution for each bin
        bin_areas = []
        bin_mb_percentage = []

        for i in np.arange(len(bin_heights) - 1, dtype=np.int):
            # indices for area interpolation
            higher_index = (np.where(run_surface_heights[1] <=
                                     bin_heights[i], 1, 0)).argmax()
            lower_index = ((np.where(run_surface_heights[1] >=
                                     bin_heights[i + 1], 1, 0)).argmin() - 1)

            tmp_area = 0
            tmp_mb_percentage = 0

            # if no grid point between indexes -> only subarea of one trapezoid
            if lower_index < higher_index:
                total_area = glacier_wide_trapezoidal_area[higher_index - 1]
                # total height of cell
                h = (run_surface_heights[1][higher_index-1] -
                     run_surface_heights[1][higher_index])

                # widths of cell to calculate subarea
                w_higher = widths[higher_index - 1]
                w_lower = widths[higher_index]

                # calculate higer subarea for subtraction
                h_diff_higher = run_surface_heights[1][higher_index -
                                                       1] - bin_heights[i]
                # convert height ratio into horizontal lenght difference
                x_diff_higher = convert_height_diff_to_x_diff(
                    h, map_dx, h_diff_higher)
                # formula from geometrical consideration of isosceles trapezoid
                subarea_higher = calc_trapezodidal_subarea(w_higher,
                                                           w_lower,
                                                           map_dx,
                                                           x_diff_higher)

                # calculate lower subarea for subtraction
                h_diff_lower = bin_heights[i+1] - \
                    run_surface_heights[1][higher_index]
                # convert height ratio into horizontal lenght difference
                x_diff_lower = convert_height_diff_to_x_diff(
                    h, map_dx, h_diff_lower)
                # formula from geometrical consideration of isosceles trapezoid
                subarea_lower = calc_trapezodidal_subarea(w_lower,
                                                          w_higher,
                                                          map_dx,
                                                          x_diff_lower)

                tmp_area = total_area - subarea_higher - subarea_lower

                fact = tmp_area / total_area
                tmp_mb_percentage += fact * \
                    glacier_mb_percentage[higher_index - 1]

            # at least one gird point between the two indices
            else:
                if higher_index != lower_index:
                    # add areas which are complete in bin using trapezoidal area
                    tmp_area = np.sum(glacier_wide_trapezoidal_area[np.arange(
                        higher_index, lower_index, dtype=np.int)])

                    # add mb_percentage from complete cells in bin
                    tmp_mb_percentage = np.sum(
                        glacier_mb_percentage[np.arange(
                            higher_index, lower_index, dtype=np.int)])

                # add area from higher portion if not at the highest point
                if (higher_index != 0):
                    # to get ratio of how much af the cell lies in bin
                    h = (run_surface_heights[1][higher_index-1] -
                         run_surface_heights[1][higher_index])
                    h_diff = bin_heights[i] - \
                        run_surface_heights[1][higher_index]

                    # convert height ratio into horizontal lenght difference
                    x_diff = convert_height_diff_to_x_diff(h, map_dx, h_diff)

                    # widths of cell to calculate subarea
                    w1 = widths[higher_index]
                    w2 = widths[higher_index - 1]

                    # formula from geometrical consideration of isosceles trapezoid
                    subarea = calc_trapezodidal_subarea(w1,
                                                        w2,
                                                        map_dx,
                                                        x_diff)
                    tmp_area += subarea

                    # add corersponding mb percentage to bin
                    fact = subarea / \
                        glacier_wide_trapezoidal_area[higher_index - 1]
                    tmp_mb_percentage += fact * \
                        glacier_mb_percentage[higher_index - 1]

                # add area from lower portion if not at the lowest point
                if (i != int(len(bin_heights) - 2)):
                    # to get ratio of how much af the cell lies in bin
                    h = (run_surface_heights[1][lower_index] -
                         run_surface_heights[1][lower_index + 1])
                    h_diff = run_surface_heights[1][lower_index] - \
                        bin_heights[i + 1]

                    # convert height difference into horizontal lenght difference
                    x_diff = convert_height_diff_to_x_diff(h, map_dx, h_diff)

                    # widths of cell to calculate subarea
                    w1 = widths[lower_index]
                    w2 = widths[lower_index + 1]

                    # formula from geometrical consideration of isosceles trapezoid
                    subarea = calc_trapezodidal_subarea(w1,
                                                        w2,
                                                        map_dx,
                                                        x_diff)
                    tmp_area += subarea

                    # add corersponding mb percentage to bin
                    fact = subarea / glacier_wide_trapezoidal_area[lower_index]
                    tmp_mb_percentage += fact * \
                        glacier_mb_percentage[lower_index]

            bin_areas = np.append(bin_areas, tmp_area)
            bin_mb_percentage = np.append(bin_mb_percentage, tmp_mb_percentage)

        # convert area to have only one digit in largest area value
        max_bin_area = np.max(bin_areas)
        for exp in [10**i for i in np.arange(10, dtype=np.int)]:
            if np.floor(max_bin_area / exp) > 0:
                if np.floor(max_bin_area / exp) < 10:
                    break

        bin_areas /= exp

        # adjust unit of x label
        if exp < 10**9:
            x1_label += ' (' + str(exp) + ' m³)'
        else:
            x1_label += ' (km³)'

        # define xlim
        xlim_area = (0, max(bin_areas)*1.05)

        # create colormap according to the bin number
        color_nr = 2 * max(mass_gain_bin_nr, mass_loss_bin_nr)
        colormap_area = hv.plotting.util.colorcet_cmap_to_palette(
            'coolwarm_r', color_nr)
        color_lim = np.max(np.abs(bin_mb_percentage))
        clim_area = (-color_lim, color_lim)

        # draw area histogram with colorcoded mass balance contribution
        mb_curve = hv.Polygons([{x1_label: [0,
                                            bin_areas[i],
                                            bin_areas[i],
                                            0],
                                 y_label: [bin_heights[i],
                                           bin_heights[i],
                                           bin_heights[i + 1],
                                           bin_heights[i + 1]],
                                 'mb_cont': bin_mb_percentage[i]}
                                for i in np.arange(len(bin_areas), dtype=np.int)],
                               kdims=[x1_label, y_label],
                               vdims='mb_cont').opts(default_tools=default_tools_geometry,
                                                     line_color='black',
                                                     cmap=colormap_area,
                                                     clim=clim_area,
                                                     xaxis='top',
                                                     xlim=xlim_area,
                                                     colorbar=True,
                                                     colorbar_opts={'title': '% MB', 'width': 10})
    else:
        colormap_area = hv.plotting.util.colorcet_cmap_to_palette(
            'coolwarm_r', 10)
        color_lim = 10
        clim_area = (-color_lim, color_lim)

        mb_curve = hv.Polygons([{x1_label: [0],
                                 y_label: [0],
                                 'mb_cont': [0]}],
                               kdims=[x1_label, y_label],
                               vdims='mb_cont').opts(default_tools=default_tools_geometry,
                                                     xaxis='top',
                                                     xlim=(0, 1),
                                                     cmap=colormap_area,
                                                     clim=clim_area,
                                                     colorbar=True,
                                                     colorbar_opts={'title': '% MB', 'width': 10})

    # add height dependent mass balance and ELA with second x axis
    mb_curve *= (hv.Curve((annual_mb, y_range_of_mb_curve),
                          x2_label,
                          y_label,
                          label=mass_balance_label
                          ).opts(default_tools=default_tools_geometry,
                                 color='black',
                                 hooks=[mb_curve_second_x_axis]) *
                 hv.Curve((x_range_of_mb_curve, np.repeat(ELA, 2)),
                          x2_label,
                          y_label,
                          label='ELA'
                          ).opts(default_tools=default_tools_geometry,
                                 line_dash='dashed',
                                 line_width=3,
                                 color='black',
                                 hooks=[mb_curve_second_x_axis]))

    mb_curve.opts(ylim=tuple(
        [y_range_of_mb_curve[0], y_range_of_mb_curve[-1]]))
    mb_curve.opts(opts.Polygons(framewise=True))

### set width curve

In [ ]:
def set_width_curve():
    global width_curve

    # set labels according to language
    x_label = geometry_plot_labels['bedrock_x_label'][language]
    y_label = geometry_plot_labels['bedrock_width_y_label'][language]

    if not np.array_equal(run_surface_heights[0], bed_h):
        old_glacier = (distance_along_glacier, -
                       glacier_outlines[0]/2, glacier_outlines[0]/2)
        new_glacier_alpha = 0.6
    else:
        old_glacier = ([], [], [])
        new_glacier_alpha = 1

    width_curve = (hv.Area(old_glacier,
                           kdims=[x_label],
                           vdims=[y_label, 'y2']
                           # label='old'
                           ).opts(default_tools=default_tools_geometry,
                                  line_dash='dashed',
                                  color='white',
                                  line_color='darkgray',
                                  alpha=0.6,
                                  line_alpha=1,
                                  line_width=3) *
                   hv.Area((distance_along_glacier, -glacier_outlines[1]/2, glacier_outlines[1]/2),
                               kdims=[x_label],
                               vdims=[y_label, 'y2']
                               # label='new'
                           ).opts(default_tools=default_tools_geometry,
                                  color='white',
                                  line_color=glacier_line_color,
                                  alpha=new_glacier_alpha,
                                  line_alpha=1,
                                  line_width=3))
    # looking for potential segments which can be colored if wanted
    index_of_segments = np.arange(np.sum(model.fls[-1].thick > 0) - 1)

    # show ice velocity in colors if wanted and there is something to show
    if show_thickness.value and index_of_segments.size != 0:
        # calculate the mean thickness between two segments
        plot_thickness = [(np.array(model.get_diagnostics()['ice_thick'])[i] +
                           np.array(model.get_diagnostics()['ice_thick'])[i + 1]) / 2
                          for i in index_of_segments]

        # define number of thickness categories
        nr_categories = np.size(colormap_h)

        # find boarders of thickness categories
        thickness_categories = np.linspace(np.min(plot_thickness),
                                           np.max(plot_thickness),
                                           nr_categories + 1,
                                           endpoint=True)

        # create array with categories number
        segment_category = np.zeros(np.size(index_of_segments))
        for category in np.arange(nr_categories):
            segment_category = np.where((plot_thickness >= thickness_categories[category]) &
                                        (plot_thickness <=
                                         thickness_categories[category + 1]),
                                        category + 1,
                                        segment_category)

        # get arguments where one category is changing to another,
        # + 1 to always get the first argument of a new category
        category_change_arg = np.argwhere(np.diff(segment_category)
                                          ).transpose().squeeze() + 1

        # add first and last argument
        category_change_arg = np.append(0, category_change_arg)
        category_change_arg = np.append(
            category_change_arg, np.size(index_of_segments))

        # define number of polygons to draw
        nr_polygons = np.size(category_change_arg) - 1

        # define x and y values of polygons and assign a first velocity to whole polygon
        x_polygons = np.empty(nr_polygons, dtype=object)
        y_polygons = np.empty(nr_polygons, dtype=object)
        thickness_polygons = np.empty(nr_polygons)

        for polygon in np.arange(nr_polygons):
            index = np.arange(category_change_arg[polygon],
                              category_change_arg[polygon + 1] + 1)
            index_r = np.arange(category_change_arg[polygon + 1],
                                category_change_arg[polygon] - 1,
                                -1)

            x_polygons[polygon] = np.append(distance_along_glacier[index],
                                            distance_along_glacier[index_r])

            y_polygons[polygon] = np.append(-glacier_outlines[1][index] / 2,
                                            glacier_outlines[1][index_r] / 2)

            thickness_polygons[polygon] = segment_category[category_change_arg[polygon]]

        width_curve *= hv.Polygons([{'x': x_polygons[i],
                                     'y': y_polygons[i],
                                     'thickness': thickness_polygons[i]}
                                    for i in np.arange(nr_polygons)],
                                   vdims='thickness').opts(default_tools=default_tools_geometry,
                                                           line_color=None,
                                                           cmap=colormap_h)
    else:
        # when there is no glacier or colors should not be shown
        # this is needed so bokeh knows about the Polygons element
        width_curve *= hv.Polygons([{'x': [0],
                                     'y': [0],
                                     'thickness': [0]}],
                                   vdims='thickness').opts(default_tools=default_tools_geometry)

    # add centerline to width _curve
    width_curve *= hv.Curve((distance_along_glacier, np.repeat(0, np.size(distance_along_glacier))),
                            x_label,
                            y_label,
                            # label='center Flowline'
                            ).opts(default_tools=default_tools_geometry,
                                   color='black',
                                   line_width=1)

    # set the ELA position and width on the x axis to follow the current glacier surface height
    set_ELA_x_position_and_width()
    ELA_x = ELA_x_position

    # get the glacier width at current ELA position
    ELA_y_min = -ELA_width / 2
    ELA_y_max = ELA_width / 2

    # add responsive ELA
    width_curve *= hv.Curve((np.repeat(ELA_x_position, 2), [ELA_y_min, ELA_y_max]),
                            x_label,
                            y_label,
                            # label='ELA'
                            ).opts(default_tools=default_tools_geometry,
                                   line_dash='dashed',
                                   line_width=3,
                                   color='black')

    width_curve.opts(ylim=(width_plot_y_min, width_plot_y_max))

### set info text

In [ ]:
def set_info_text(stage='new model'):
    global info_text
    global last_info_text_stage

    # is needed when language is switched
    last_info_text_stage = stage

    text_color = 'black'

    if stage == 'new model':
        text = geometry_plot_labels['info_text_new_model'][language]
    elif stage == 'switched off':
        text = geometry_plot_labels['info_text_switched_off'][language]
        text_color = 'red'
    elif stage == 'switched on':
        text = geometry_plot_labels['info_text_switched_on'][language]
        text_color = 'darkgreen'
    else:
        if stage == 'running model':
            text = geometry_plot_labels['info_text_running_model'][language]
        elif stage == 'equilibrium':
            text = geometry_plot_labels['info_text_equilibrium'][language]
        elif stage == 'stopped model':
            text = geometry_plot_labels['info_text_stopped_model'][language]
        elif stage == 'aborted model':
            text = geometry_plot_labels['info_text_aborted_model'][language]
        elif stage == 'outside domain':
            text = geometry_plot_labels['info_text_outside_domain'][language]

        text += geometry_plot_labels['info_text_statistics'][language]
        text = text.format(model.yr,
                           model.length_m / 1000,
                           model.area_km2,
                           model.volume_km3,
                           np.max(model.get_diagnostics()['ice_thick']),
                           np.max(model.get_diagnostics()[
                                  'ice_velocity']) * cfg.SEC_IN_YEAR,
                           calculate_AAR(),
                           glacier_wide_mb)

    info_text = hv.Text(0, 0, text).opts(default_tools=['save', 'reset'],
                                         show_frame=False,
                                         text_baseline='middle',
                                         text_align='center',
                                         text_font_size=infotext_fontsize,
                                         text_font='courier',
                                         text_color=text_color,
                                         xaxis=None,
                                         yaxis=None)

### help functions for timeseries curves

In [ ]:
from app_text import timeseries_plot_labels

In [ ]:
def get_timeseries_y_value_and_label(option):
    # returns python array, first entry y values, second entry y label
    if option is panel_text['timeseries_axis_options'][language][0]:
        return [glacier_volume, timeseries_plot_labels['y_label_volume'][language]]

    elif option is panel_text['timeseries_axis_options'][language][1]:
        return [glacier_area, timeseries_plot_labels['y_label_area'][language]]

    elif option is panel_text['timeseries_axis_options'][language][2]:
        return [glacier_length, timeseries_plot_labels['y_label_length'][language]]

    elif option is panel_text['timeseries_axis_options'][language][3]:
        return [glacier_max_velocity, timeseries_plot_labels['y_label_max_velocity'][language]]

    elif option is panel_text['timeseries_axis_options'][language][4]:
        return [glacier_max_thickness, timeseries_plot_labels['y_label_max_thickness'][language]]

    elif option is panel_text['timeseries_axis_options'][language][5]:
        return [glacier_max_ice_flux, timeseries_plot_labels['y_label_max_ice_flux'][language]]
    elif option is panel_text['timeseries_axis_options'][language][6]:
        return [glacier_AAR, timeseries_plot_labels['y_label_AAR'][language]]
    else:
        return [glacier_volume, timeseries_plot_labels['y_label_volume'][language]]

### set timeseries top curve

In [ ]:
def set_timeseries_top_curve():
    global timeseries_top_curve

    # set labels according to language
    x_label = timeseries_plot_labels['x_label'][language]
    y_value, y_label = get_timeseries_y_value_and_label(timeseries_top.value)

    timeseries_top_curve = hv.Curve((glacier_time, y_value),
                                    x_label,
                                    y_label
                                    ).opts(default_tools=default_tools_timeseries,
                                           active_tools=default_active_tools_timeseries)

    # draws 11 helplines to seperate the sections
    for i in np.arange(11):
        # looks if the i'th line is needed, else the line will be drawn at zero
        try:
            vlin_pos = timeseries_vlin_pos[i]
        except IndexError:
            vlin_pos = 0

        # actual add the vertical lines
        timeseries_top_curve *= hv.VLine(vlin_pos).opts(default_tools=default_tools_timeseries,
                                                        active_tools=default_active_tools_timeseries,
                                                        color='black')

    # set default range to (0, 1)
    if y_value == [0]:
        timeseries_top_curve.opts(xlim=(0, 1),
                                  ylim=(0, 1))
    else:
        timeseries_top_curve.opts(xlim=(np.nan, np.nan),
                                  ylim=(np.nan, np.nan))

    # add grid to plot
    grid_style = {'xgrid_line_color': 'white', 'ygrid_line_color': None,
                  'grid_line_width': 2}
    timeseries_top_curve.opts(show_grid=True,
                              gridstyle=grid_style,
                              bgcolor='#ebebeb')

    # is needed for dynamical axes
    timeseries_top_curve.opts(opts.Curve(framewise=True))

### set timeseries middle curve

In [ ]:
def set_timeseries_middle_curve():
    global timeseries_middle_curve

    # set labels according to language
    x_label = timeseries_plot_labels['x_label'][language]
    y_value, y_label = get_timeseries_y_value_and_label(
        timeseries_middle.value)

    timeseries_middle_curve = hv.Curve((glacier_time, y_value),
                                       x_label,
                                       y_label
                                       ).opts(default_tools=default_tools_timeseries,
                                              active_tools=default_active_tools_timeseries)

    # draws 11 helplines to seperate the sections
    for i in np.arange(11):
        # looks if the i'th line is needed, else the line will be drawn at zero
        try:
            vlin_pos = timeseries_vlin_pos[i]
        except IndexError:
            vlin_pos = 0

        # actual add the vertical lines
        timeseries_middle_curve *= hv.VLine(vlin_pos).opts(default_tools=default_tools_timeseries,
                                                           active_tools=default_active_tools_timeseries,
                                                           color='black')

    # set default range to (0, 1)
    if y_value == [0]:
        timeseries_middle_curve.opts(xlim=(0, 1),
                                     ylim=(0, 1))
    else:
        timeseries_middle_curve.opts(xlim=(np.nan, np.nan),
                                     ylim=(np.nan, np.nan))

    # add grid to plot
    grid_style = {'xgrid_line_color': 'white', 'ygrid_line_color': None,
                  'grid_line_width': 2}
    timeseries_middle_curve.opts(show_grid=True,
                                 gridstyle=grid_style,
                                 bgcolor='#ebebeb')

    # is needed for dynamical axes
    timeseries_middle_curve.opts(opts.Curve(framewise=True))

### set timeseries bottom curve

In [ ]:
def set_timeseries_bottom_curve():
    global timeseries_bottom_curve

    # set labels according to language
    x_label = timeseries_plot_labels['x_label'][language]
    y_value, y_label = get_timeseries_y_value_and_label(
        timeseries_bottom.value)

    timeseries_bottom_curve = hv.Curve((glacier_time, y_value),
                                       x_label,
                                       y_label
                                       ).opts(default_tools=default_tools_timeseries,
                                              active_tools=default_active_tools_timeseries)

    # variable to count how many 'empty' sections
    empty_sections_count = 0

    # draws 11 helplines to seperate the sections and add the section number
    for i in np.arange(11):
        # set the section number to default (if not needed)
        section_number_text = ' '
        section_number_x = 0
        section_number_y = 0

        # looks if the i'th line and text is needed
        try:
            vlin_pos = timeseries_vlin_pos[i]

            # looks if vertical line position is not zero and
            # then set the section number text and position
            if vlin_pos != 0:
                section_number_text = ('# ' + str(i - 1))
                section_number_x = ((timeseries_vlin_pos[i] -
                                     timeseries_vlin_pos[i - 1]) / 2 +
                                    timeseries_vlin_pos[i - 1])
                section_number_y = np.max(y_value) / 4
        except IndexError:
            vlin_pos = 0

        if vlin_pos == 0:
            empty_sections_count += 1

        # actual add the vertical lines
        timeseries_bottom_curve *= hv.VLine(vlin_pos).opts(default_tools=default_tools_timeseries,
                                                           active_tools=default_active_tools_timeseries,
                                                           color='black')

        # actual add the section number
        timeseries_bottom_curve *= hv.Text(section_number_x,
                                           section_number_y,
                                           section_number_text
                                           ).opts(default_tools=default_tools_timeseries,
                                                  active_tools=default_active_tools_timeseries)

    # set default range to (0, 1)
    if y_value == [0]:
        timeseries_bottom_curve.opts(xlim=(0, 1),
                                     ylim=(0, 1))
    else:
        timeseries_bottom_curve.opts(xlim=(np.nan, np.nan),
                                     ylim=(np.nan, np.nan))

    # add grid to plot
    grid_style = {'xgrid_line_color': 'white', 'ygrid_line_color': None,
                  'grid_line_width': 2}
    timeseries_bottom_curve.opts(show_grid=True,
                                 gridstyle=grid_style,
                                 bgcolor='#ebebeb')

    # is needed for dynamical axes
    timeseries_bottom_curve.opts(opts.Curve(framewise=True))

### set timeseries table

In [ ]:
from app_text import timeseries_table_labels

In [ ]:
def set_timeseries_table():
    global timeseries_table

    table_heading_1 = timeseries_table_labels['table_heading_1'][language]
    table_heading_2 = timeseries_table_labels['table_heading_2'][language]

    # are needed to use different languages
    sliding_table = np.where(table_values['sliding'],
                             timeseries_table_labels['yes_label'][language],
                             timeseries_table_labels['no_label'][language])
    equilibrium_table = np.where(table_values['equilibrium'],
                                 timeseries_table_labels['yes_label'][language],
                                 timeseries_table_labels['no_label'][language])
    outside_domain_table = np.where(table_values['outside_domain'],
                                    timeseries_table_labels['yes_label'][language],
                                    timeseries_table_labels['no_label'][language])

    if len(table_values['bedrock_profil']) == 0:
        bedrock_profil_table = []
    else:
        bedrock_profil_table = np.repeat(panel_text['bed_rock_profil_values'][language][table_values['bedrock_profil'][0]],
                                         len(table_values['bedrock_profil']))

    if len(table_values['width']) == 0:
        bedrock_width_table = []
    else:
        bedrock_width_table = np.repeat(panel_text['bed_rock_width_values'][language][table_values['width'][0]],
                                        len(table_values['width']))

    if len(table_values['glen_a']) == 0:
        glen_a_table = []
    else:
        glen_a_table = np.repeat('', len(table_values['glen_a']))
        glen_a_table = np.where(np.array(table_values['glen_a']) == 0,
                                panel_text['glens_creep_parameter_values'][language][0],
                                glen_a_table)
        glen_a_table = np.where(np.array(table_values['glen_a']) == 1,
                                panel_text['glens_creep_parameter_values'][language][1],
                                glen_a_table)
        glen_a_table = np.where(np.array(table_values['glen_a']) == 2,
                                panel_text['glens_creep_parameter_values'][language][2],
                                glen_a_table)

    timeseries_table = hv.Table((bedrock_profil_table,
                                 table_values['slope'],
                                 bedrock_width_table,
                                 table_values['ELA'],
                                 table_values['mb_gradient'],
                                 glen_a_table,
                                 sliding_table,
                                 table_values['years'],
                                 equilibrium_table,
                                 outside_domain_table),
                                table_heading_1,
                                table_heading_2
                                )

## Define function to run model

### help functions for run model function

In [ ]:
def set_model_options():
    global dyears
    global max_calc_years

    dyears = dyears_model_slider.value
    max_calc_years = max_calc_years_slider.value

In [ ]:
def run_and_update_model_for_10yrs():
    global model
    global run_surface_heights
    global glacier_outlines
    global timeseries_table_change_index
    global timeseries_figure_change_index
    global geometry_figure_change_index

    start_yr = model.yr

    while model.yr - start_yr < 10:
        model.run_until(model.yr + dyears)

        # update geometry plot if this is wanted
        if show_geometry_plot.value:
            # save new surface/outline and show it on plot
            run_surface_heights[1] = np.array(
                model.fls[-1].surface_h, ndmin=2)
            glacier_outlines[1] = get_glacier_outline()

            # calculate glacier wide mass balance
            set_glacier_wide_mb()

            # set new stage of all subfigures
            set_info_text(stage='running model')
            set_glacier_height_curve()
            set_width_curve()
            set_mb_curve()

            # trigger event
            with unlocked():
                geometry_figure.event(change=geometry_figure_change_index)
            geometry_figure_change_index = not geometry_figure_change_index

        # save values for timeseries plot
        max_u = np.max(model.get_diagnostics()[
                       'ice_velocity']) * cfg.SEC_IN_YEAR
        max_h = np.max(model.get_diagnostics()['ice_thick'])
        max_f = np.max(model.get_diagnostics()[
                       'ice_flux']) * cfg.SEC_IN_YEAR
        glacier_volume.append(model.volume_km3)
        glacier_area.append(model.area_km2)
        glacier_length.append(model.length_m/1000)
        glacier_max_velocity.append(max_u)
        glacier_max_thickness.append(max_h)
        glacier_max_ice_flux.append(max_f)
        glacier_AAR.append(calculate_AAR())
        glacier_time.append(glacier_time[-1] + dyears)

### actual run model function

In [ ]:
def run_the_model(stage='beginner mode'):
    global model
    global run_surface_heights
    global glacier_outlines
    global timeseries_table_change_index
    global timeseries_figure_change_index
    global geometry_figure_change_index

    # set model options according to the current slider values
    set_model_options()

    if stage == 'beginner mode':
        # initialize a new model
        init_model()

    elif stage == 'advance equilibrium' or stage == 'advance years':
        # initialize model with old extend and new parameters
        set_mb_model()
        set_glen_a()
        set_fs()
        model = FlowlineModel(model.fls[-1], mb_model=mb_model, y0=0.,
                              glen_a=glen_a, fs=fs, check_for_boundaries=False,
                              mb_elev_feedback='always')

        # determine glacier state at starting point
        run_surface_heights = np.repeat(
            np.array(model.fls[-1].surface_h, ndmin=2), 2, axis=0)
        glacier_outlines = np.repeat(get_glacier_outline(), 2, axis=0)

    # variable to check if glacier is growing outside the domain
    outside_domain_flag = False

    # variable to check if glacier in equilibrium
    equilibrium_flag = False

    # defines maximum calculaction time of model according to button click
    if stage in ['beginner mode', 'advance equilibrium']:
        maximum_model_year = max_calc_years
    elif stage == 'advance years':
        maximum_model_year = run_model_years.value

    while model.yr < maximum_model_year:
        # stop model run if glacier is running outside the domain (if option is choosed)
        if stop_if_outside_domain.value:
            if np.sum(model.fls[-1].thick > 0) == nx:
                outside_domain_flag = True
                break

        # check for equilibrium only every 10 years, similar OGGM FowlineModel.run_until_equilibrium
        v_bef = model.volume_m3
        run_and_update_model_for_10yrs()
        v_af = model.volume_m3
        t_rate = np.abs(v_af - v_bef) / v_bef
        # threshold for the volume change during 10 years for equilibrium 0.001
        if t_rate < 0.001:
            equilibrium_flag = True
            # end model run if you are searching for equilibrium state
            if stage in ['beginner mode', 'advance equilibrium']:
                break

    # check if model was aborted due to the maximum calculation time
    if stage in ['beginner mode', 'advance equilibrium'] and (model.yr >= max_calc_years):
        reached_max_calc_year_flag = True
    else:
        reached_max_calc_year_flag = False

    # update geometry plot at end of model run

    # save new surface/outline and show it on plot
    run_surface_heights[1] = np.array(
        model.fls[-1].surface_h, ndmin=2)
    glacier_outlines[1] = get_glacier_outline()

    # calculate glacier wide mass balance
    set_glacier_wide_mb()

    # set new stage of all subfigures
    if outside_domain_flag:
        set_info_text(stage='outside domain')
    elif reached_max_calc_year_flag:
        set_info_text(stage='aborted model')
    elif equilibrium_flag:
        set_info_text(stage='equilibrium')
    else:
        set_info_text(stage='stopped model')
    set_glacier_height_curve()
    set_width_curve()
    set_mb_curve()

    # set new stage of all subfigures
    with unlocked():
        geometry_figure.event(change=geometry_figure_change_index)
    geometry_figure_change_index = not geometry_figure_change_index

    # update timeseries plot at end of model run

    # store time for vertical lines in volume and length plot
    timeseries_vlin_pos.append(glacier_time[-1])

    # set volume, area and length curves with new vertical line
    set_timeseries_top_curve()
    set_timeseries_middle_curve()
    set_timeseries_bottom_curve()
    with unlocked():
        timeseries_figure.event(change=timeseries_figure_change_index)
    timeseries_figure_change_index = not timeseries_figure_change_index

    # store values for table
    table_values['years'].append(int(model.yr))
    table_values['ELA'].append(ELA_height.value)
    table_values['mb_gradient'].append(mb_gradient.value)
    table_values['equilibrium'].append(equilibrium_flag)
    table_values['outside_domain'].append(outside_domain_flag)

    if glens_creep_parameter.value is panel_text['glens_creep_parameter_values'][language][0]:
        table_values['glen_a'].append(0)
    elif glens_creep_parameter.value is panel_text['glens_creep_parameter_values'][language][1]:
        table_values['glen_a'].append(1)
    elif glens_creep_parameter.value is panel_text['glens_creep_parameter_values'][language][2]:
        table_values['glen_a'].append(2)

    if sliding_parameter.value is panel_text['sliding_parameter_values'][language][1]:
        table_values['sliding'].append(True)
    else:
        table_values['sliding'].append(False)

    # can use tmporary values because geometry only changed when new model is initialized
    table_values['bedrock_profil'].append(
        tmp_table_values['bedrock_profil'])
    table_values['slope'].append(tmp_table_values['slope'])
    table_values['width'].append(tmp_table_values['width'])

    # set the new values in the table
    set_timeseries_table()

    # force the table to refresh
    with unlocked():
        timeseries_table_DynamicMap.event(
            change=timeseries_table_change_index)
    timeseries_table_change_index = not timeseries_table_change_index

## Functions for dynamic maps (arrangement of subplots)

This functions define how the subplots will be arranged.

### change geometry figure

In [ ]:
def change_geometry_figure(change=True):
    return ((glacier_height_curve * bed_rock_height_curve).opts(legend_position='bottom_left',
                                                                legend_cols=2,
                                                                xaxis='top',
                                                                bgcolor='lightblue',
                                                                responsive=True,
                                                                aspect=geometry_figure_scale*1.2) +
            (mb_curve).opts(legend_position='bottom_right',
                            xaxis='top',
                            yaxis='right',
                            bgcolor='lightgray',
                            responsive=True,
                            aspect=geometry_figure_scale*1.2) +
            (bed_rock_width_curve * width_curve).opts(responsive=True,
                                                      aspect=4,
                                                      yformatter='%4.0f',
                                                      bgcolor='darkgray',
                                                      labelled=['y']) +
            (info_text).opts(responsive=True,
                             aspect=4,
                             margin=[0, 1, 0, 1])
            ).opts(sizing_mode='stretch_both',
                   toolbar='right').cols(2)

### change timeseries figure

In [ ]:
def change_timeseries_figure(change=True):
    return ((timeseries_top_curve).opts(responsive=True,
                                        xaxis='top',
                                        aspect=7.1) +
            (timeseries_middle_curve).opts(responsive=True,
                                           aspect=9.5,
                                           labelled=['y'],
                                           xaxis=None) +
            (timeseries_bottom_curve).opts(responsive=True,
                                           labelled=['y'],
                                           aspect=8.4)
            ).opts(sizing_mode='stretch_both',
                   toolbar='right').cols(1)

### change timeseries table

In [ ]:
def timeseries_table_hook(plot, element):
    plot.handles['table'].sizing_mode = 'stretch_width'


def change_timeseries_table(change=True):
    return timeseries_table.opts(height=110,
                                 hooks=[timeseries_table_hook])

## streamers and dynamic maps

before creating the dynamic maps initialise the model and all subplots with the default values

In [ ]:
init_model()

dynamic map and stream for geometry plot

In [ ]:
geometry_figure_stream = Stream.define('geometry_figure_stream', change=False)

geometry_figure = hv.DynamicMap(
    change_geometry_figure, streams=[geometry_figure_stream()])

dynamic maps and streams for timeseries plot (figure and table)

In [ ]:
timeseries_figure_stream = Stream.define(
    'timeseries_figure_stream', change=False)

timeseries_figure = hv.DynamicMap(
    change_timeseries_figure, streams=[timeseries_figure_stream()])

In [ ]:
timeseries_table_stream = Stream.define(
    'timeseries_table_stream', change=False)

timeseries_table_DynamicMap = hv.DynamicMap(
    change_timeseries_table, streams=[timeseries_table_stream()])

## put figures together in tab menu

In [ ]:
from app_text import figure_tab_headings

In [ ]:
figures = pn.Tabs(('', []),
                  ('', []),
                  tabs_location='above',
                  sizing_mode='stretch_both')

figures_tab1 = geometry_figure
figures_tab2 = pn.Column(timeseries_figure,
                         pn.Row(timeseries_table_DynamicMap,
                                height_policy='max'),
                         sizing_mode='stretch_both')


# is necessary to change tabs title language
def set_figures_tabs():
    figures.clear()

    figures.append((figure_tab_headings['geometry_plot'][language],
                    figures_tab1))

    figures.append((figure_tab_headings['timeseries_plot'][language],
                    figures_tab2))


set_figures_tabs()

## App

In [ ]:
pn.Column(header,
          pn.Row(menu,
                 figures),
          sizing_mode='stretch_both').servable(title='Glacier Simulator')

As long as you are running this notebook "live" (in Jupyter, not viewing a website or a static copy), the above notebook cell should contain the fully operational dashboard here in the notebook. You can also launch the dashboard at a separate port that shows up in a new browser tab, either by changing .servable() to .show() above and re-executing that cell, or by leaving the cell as it is and running bokeh serve --show simulator.ipynb.